<a href="https://colab.research.google.com/github/ML-HW-SYS/a3-anya-23-ct/blob/main/4_gemm_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEMM on GPU

## 1. Set-up

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [17]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

# %mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
# !git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

/content/gdrive/MyDrive/ece5545
/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
Already on 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), 380 bytes | 9.00 KiB/s, done.
From https://github.com/ML-HW-SYS/a3-anya-23-ct
   540dedd..2f1d79c  main       -> origin/main
Updating 540dedd..2f1d79c
Fast-forward
 src/ops.py | 7 +------
 1 file changed, 1 insertion(+), 6 deletions(-)
/content/gdrive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [5]:
!ls {PROJECT_ROOT}

1_conv1d_cpu.ipynb  3-conv1d_fpga.ipynb  5_conv2d_dw_gpu.ipynb	README.md  tests
2_conv1d_gpu.ipynb  4_gemm_gpu.ipynb	 leaderboard_id.txt	src


## 2. Install TVM

In [6]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.6/428.6 MB 2.3 MB/s eta 0:00:00


## 3. Check the implementation of `make_gemm_gpu_scheduler` function in `src.ops`

The function implements General Matrix Multiply (GEMM) on GPU. You should use TVM to optimize it.

Let $A \in \mathbb{R}^{m \times k}$, $W \in \mathbb{R}^{k \times n}$, and $B \in \mathbb{R}^{m \times n}$, then
$$
B = A \times W
$$
Please see the numpy matmul function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html).

The `make_gemm_gpu_scheduler` takes $m$, $k$, and $n$. The first matrix is $m \times k$, the second matrix is $k \times n$, and the output matrix is $m \times n$.

The function returns both the TVM scheduler and the TVM opterator for
1. Input $a$
2. Input $w$
3. Output $b$

The scheduler should be able to used to build a function with signature $func(a, w, b)$.
Please see the following cells for usage.

In [ ]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_gemm_gpu_scheduler

M = 1024
N = 512
K = 2048
dtype = 'float32'
a_np = np.random.rand(M, K).astype(dtype)
w_np = np.random.rand(K, N).astype(dtype)
b_np = np.matmul(a_np, w_np)

s, A, W, B = make_gemm_gpu_scheduler(M, K, N)
func = tvm.build(s, [A, W, B], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M, N), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"GEMM TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [[498.6206  497.29578 512.71533 ... 507.04666 501.69836 496.20288]
 [505.1839  502.87903 511.18536 ... 505.7803  503.62958 501.30338]
 [494.72662 507.2359  515.82007 ... 505.81396 496.44647 503.10767]
 ...
 [497.98547 507.85663 517.41156 ... 504.03702 494.86145 504.2512 ]
 [497.23294 506.19016 510.91302 ... 502.3822  493.55756 498.72968]
 [506.4279  512.9241  521.8143  ... 509.13065 507.63358 502.62515]]
Output: [[498.61993 497.29593 512.71466 ... 507.04657 501.6985  496.20288]
 [505.18433 502.8787  511.18506 ... 505.78067 503.62964 501.30322]
 [494.72662 507.23544 515.8204  ... 505.81433 496.44653 503.10785]
 ...
 [497.9852  507.8573  517.4118  ... 504.03745 494.8614  504.2511 ]
 [497.23328 506.19003 510.9128  ... 502.38232 493.55713 498.73007]
 [506.42764 512.92377 521.8146  ... 509.13068 507.63345 502.62524]]
GEMM TVM: 86.403068 ms


In [ ]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 2048), "float32"), B: T.Buffer((2048, 512), "float32"), C: T.Buffer((1024, 512), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        blockIdx_y = T.launch_thread("blockIdx.y", 1024)
        blockIdx_x = T.launch_thread("blockIdx.x", 512)
        C_1 = T.Buffer((524288,), data=C.data)
        C_1[blockIdx_y * 512 + blockIdx_x] = T.float32(0)
        for k in range(2048):
            A_1 = T.Buffer((2097152,), data=A.data)
            B_1 = T.Buffer((1048576,), data=B.data)
            C_1[blockIdx_y * 512 + blockIdx_x] = C_1[blockIdx_y * 512 + blockIdx_x] + A_1[blockIdx_y * 2048 + k] * B_1[k * 512 + blockIdx_x]


In [ ]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_gemm_gpu.py

/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-anya-23-ct
plugins: anyio-3.7.1
collected 20 items                                                                                 

tests/test_gemm_gpu.py ....................                                                  [100%]

======================================= 20 passed in 25.48s ========================================


After Optimisation

In [18]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_gemm_gpu_scheduler

M = 1024
N = 512
K = 2048
dtype = 'float32'
a_np = np.random.rand(M, K).astype(dtype)
w_np = np.random.rand(K, N).astype(dtype)
b_np = np.matmul(a_np, w_np)

s, A, W, B = make_gemm_gpu_scheduler(M, K, N)
func = tvm.build(s, [A, W, B], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M, N), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"GEMM TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [[519.0353  509.1373  503.14197 ... 512.7716  501.28372 499.01343]
 [516.383   500.28494 503.69202 ... 503.76196 487.74896 495.8604 ]
 [530.76135 519.27655 514.59314 ... 523.7985  507.06186 505.90936]
 ...
 [519.6876  501.01474 504.2394  ... 502.29172 496.34    495.0815 ]
 [519.18134 504.7581  503.9147  ... 512.88446 488.27893 504.00406]
 [522.40643 501.3681  507.99384 ... 517.32556 499.3972  503.78522]]
Output: [[519.03546 509.13724 503.1417  ... 512.7712  501.2839  499.01346]
 [516.3834  500.28485 503.69168 ... 503.76193 487.74887 495.86017]
 [530.7618  519.2765  514.5932  ... 523.798   507.06125 505.90924]
 ...
 [519.6876  501.01517 504.23926 ... 502.29147 496.34003 495.0814 ]
 [519.1815  504.75848 503.91498 ... 512.88446 488.27884 504.0041 ]
 [522.40674 501.36786 507.99387 ... 517.32544 499.39725 503.78494]]
GEMM TVM: 6.373375 ms


In [19]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 2048), "float32"), B: T.Buffer((2048, 512), "float32"), C: T.Buffer((1024, 512), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        blockIdx_y = T.launch_thread("blockIdx.y", 32)
        blockIdx_x = T.launch_thread("blockIdx.x", 16)
        threadIdx_y = T.launch_thread("threadIdx.y", 32)
        threadIdx_x = T.launch_thread("threadIdx.x", 32)
        C_1 = T.Buffer((524288,), data=C.data)
        C_1[blockIdx_y * 16384 + threadIdx_y * 512 + blockIdx_x * 32 + threadIdx_x] = T.float32(0)
        for k in range(2048):
            A_1 = T.Buffer((2097152,), data=A.data)
            B_1 = T.Buffer((1048576,), data=B.data)
            C_1[blockIdx_y * 16384 + threadIdx_y * 512 + blockIdx_x * 32 + threadIdx_x] = C_1[blockIdx_y * 16384 + threadIdx_y * 512 + blockIdx_x * 32 + threadIdx_x] 

In [20]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_gemm_gpu.py

/content/gdrive/MyDrive/ece5545/a3-anya-23-ct
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-anya-23-ct
plugins: anyio-3.7.1
collected 20 items                                                                                 

tests/test_gemm_gpu.py ....................                                                  [100%]

======================================= 20 passed in 17.94s ========================================
